# Setup & Data Initialization

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from collections import Counter
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import time



# Model Loading

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

MODEL_PATH = '/content/drive/MyDrive/model.h5'
CLASS_NAMES = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
TEST_FOLDER_PATH = '/content/drive/MyDrive/test_images'

# Pass the MobileNetV2 version of preprocess_input
model2 = load_model(MODEL_PATH, custom_objects={'preprocess_input': preprocess_input})

# Experimental Inference Methods (Legacy)

In [ ]:
# --- LEGACY METHODS (DEPRECATED) ---
# NOTE: The following Pyramid methods were explored to improve local feature extraction.
# REASON FOR DISCARD: While accurate for specific patches, they introduced a 400% - 600%
# increase in latency (O(N) complexity) which is unsuitable for real-time edge deployment.
# We maintain this code for research documentation only.

def method_2_pyramid(img_path):
    original_img = Image.open(img_path).convert('RGB')
    target_size = 800
    w, h = original_img.size
    ratio = target_size / max(w, h)
    working_img = original_img.resize((int(w * ratio), int(h * ratio)), Image.Resampling.LANCZOS)

    votes = []
    scales = [1.0, 0.75]
    patch_size = 150
    stride = 100

    for scale in scales:
        sw = int(working_img.size[0] * scale)
        sh = int(working_img.size[1] * scale)
        scaled_img = working_img.resize((sw, sh), Image.Resampling.LANCZOS)

        for y in range(0, sh - patch_size + 1, stride):
            for x in range(0, sw - patch_size + 1, stride):
                patch = scaled_img.crop((x, y, x + patch_size, y + patch_size))
                patch_arr = np.array(patch).astype('float32')

                patch_arr = np.expand_dims(patch_arr, axis=0)

                preds = model2.predict(patch_arr, verbose=0)[0]
                idx = np.argmax(preds)
                conf = preds[idx]

                if conf > 0.45:
                    votes.append(CLASS_NAMES[idx])

    return Counter(votes).most_common(1)[0][0] if votes else "Uncertain"

def method_3_weighted_pyramid(img_path):
    """V2 Update: prioritizes 'certain' patches over 'confused' ones."""
    original_img = Image.open(img_path).convert('RGB')
    target_size = 800
    w, h = original_img.size
    ratio = target_size / max(w, h)
    working_img = original_img.resize((int(w * ratio), int(h * ratio)), Image.Resampling.LANCZOS)

    # Initialize weights for all classes
    weighted_votes = {name: 0.0 for name in CLASS_NAMES}

    scales = [1.0, 0.75]
    patch_size = 150
    stride = 100

    for scale in scales:
        sw, sh = int(working_img.size[0] * scale), int(working_img.size[1] * scale)
        scaled_img = working_img.resize((sw, sh), Image.Resampling.LANCZOS)

        for y in range(0, sh - patch_size + 1, stride):
            for x in range(0, sw - patch_size + 1, stride):
                patch = scaled_img.crop((x, y, x + patch_size, y + patch_size))
                patch_arr = np.array(patch).astype('float32')
                patch_arr = np.expand_dims(patch_arr, axis=0)

                preds = model2.predict(patch_arr, verbose=0)[0]
                idx = np.argmax(preds)
                conf = preds[idx]

                # V2 CORE LOGIC: Square the confidence to give certain patches more 'weight'
                # This ensures one 90% forest patch beats two 46% building patches
                weighted_votes[CLASS_NAMES[idx]] += (conf ** 2)

    return max(weighted_votes, key=weighted_votes.get)




def method_4_edge_preserved_resize(img_path):
    """
    V4 CHAMPION: Smart Downsampling.
    Preserves edges while reducing to 150x150 for the Intel CNN.
    """
    # 1. Load image (using cv2 for speed)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # 2. Apply Bilateral Filter (Edge-Preserving Smoothing)
    # d=9: neighborhood diameter, sigmaColor/Space=75: color/coordinate influence
    smooth = cv2.bilateralFilter(img, d=9, sigmaColor=75, sigmaSpace=75)

    # 3. Downsample to 150x150 using INTER_AREA
    # This is mathematically better for downsampling than standard linear scaling.
    low_res = cv2.resize(smooth, (150, 150), interpolation=cv2.INTER_AREA)

    # 4. Final Sharpening (Unsharp Mask)
    # This makes edges 'pop' specifically for the CNN features.
    gaussian_blur = cv2.GaussianBlur(low_res, (0, 0), 2.0)
    sharpened = cv2.addWeighted(low_res, 1.5, gaussian_blur, -0.5, 0)

    # Predict using the specialized Intel model
    arr = np.expand_dims(sharpened.astype('float32'), axis=0)
    preds = model2.predict(arr, verbose=0)[0]

    return CLASS_NAMES[np.argmax(preds)]


# Optimized Inference Engines

In [ ]:
# --- THE WORKING METHODS ---

def method_1_baseline(img_path):
    img = Image.open(img_path).convert('RGB').resize((150, 150))
    arr = np.array(img).astype('float32')
    preds = model2.predict(np.expand_dims(arr, axis=0), verbose=0)[0]
    return CLASS_NAMES[np.argmax(preds)]

def method_5_fast_ensemble(img_path):
    """
    V5 CHAMPION: Optimized for production.
    Loads image once, branches into two streams, and soft-votes.
    """
    # Load once
    img_bgr = cv2.imread(img_path)
    if img_bgr is None: return "Error"
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)

    # STREAM 1: Baseline (Global Context)
    low_res_raw = cv2.resize(img_rgb, (150, 150), interpolation=cv2.INTER_AREA)
    prob_baseline = model2.predict(np.expand_dims(low_res_raw.astype('float32'), 0), verbose=0)[0]

    # STREAM 2: Smart (Local Features)
    # Re-using your logic but without re-reading from disk
    mid_res = cv2.resize(img_rgb, (300, 300), interpolation=cv2.INTER_AREA)
    smooth = cv2.bilateralFilter(mid_res, d=5, sigmaColor=50, sigmaSpace=50)

    lab = cv2.cvtColor(smooth, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    enhanced = cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2RGB)

    low_res_smart = cv2.resize(enhanced, (150, 150), interpolation=cv2.INTER_AREA)
    prob_smart = model2.predict(np.expand_dims(low_res_smart.astype('float32'), 0), verbose=0)[0]

    # FINAL VOTE
    final_probs = (prob_baseline + prob_smart) / 2
    return CLASS_NAMES[np.argmax(final_probs)]







# Performance Benchmarking

## Comparing all methods

In [ ]:
results = []
for filename in sorted(os.listdir(TEST_FOLDER_PATH)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(TEST_FOLDER_PATH, filename)

        # --- Baseline (Standard Resize) ---
        start_b = time.time()
        res_b = method_1_baseline(path)
        time_b = time.time() - start_b

         # --- Test CNN (Pyramid Model) ---
        start_pyramid = time.time()
        res_pyramid = method_2_pyramid(path)
        time_pyramid = time.time() - start_pyramid

        # --- Test CNN (Specialized Model) ---
        start_Wcnn = time.time()
        res_Wcnn = method_3_weighted_pyramid(path)
        time_Wcnn = time.time() - start_Wcnn

        # --- V4 (Smart Edge-Preserved Downsample) ---
        start_v4 = time.time()
        res_v4 = method_4_edge_preserved_resize(path)
        time_v4 = time.time() - start_v4

        # --- V5 (Smart Ensemble) ---
        start_v5 = time.time()
        res_v5 = method_5_fast_ensemble(path)
        # FIX: Corrected time calculation to use start_v5
        time_v5 = time.time() - start_v5

        results.append({
            "File": filename,
            "Baseline (Std)": res_b,
            "Pyramid": res_pyramid,
            "Weighted Pyramid": res_Wcnn,
            "Smart (V4)": res_v4,
            "Smart (V5)": res_v5,
            "Baseline Time": f"{time_b:.3f}s",
            "Pyramid Time": f"{time_pyramid:.3f}s",
           "Weighted Time": f"{time_Wcnn:.3f}s",
            "Smart v4 Time": f"{time_v4:.3f}s",
             "Smart Time": f"{time_v5:.3f}s"
        })

df_final = pd.DataFrame(results)
print(df_final.to_string(index=False))

                    File Baseline (Std)   Pyramid Weighted Pyramid Smart (V4) Smart (V5) Baseline Time Pyramid Time Weighted Time Smart v4 Time Smart Time
           building1.jpg      buildings buildings        buildings  buildings  buildings        3.636s       3.072s        4.202s        2.788s     0.832s
           building2.jpg      buildings buildings        buildings  buildings  buildings        1.177s       3.858s        4.588s        3.589s     0.962s
           building3.jpg      buildings buildings        buildings  buildings  buildings        1.346s       3.906s        4.896s        5.437s     1.603s
           building4.jpg      buildings buildings        buildings  buildings  buildings        0.933s       4.728s        3.599s        2.376s     0.605s
           building5.jpg      buildings buildings        buildings  buildings  buildings        0.877s       4.416s        3.184s        2.461s     0.774s
           building6.jpg      buildings buildings        buildings  bu

In [ ]:
output_csv_path = 'prediction_results.csv'
df_final.to_csv(output_csv_path, index=False)
print(f"Results saved to {output_csv_path}")

Results saved to prediction_results.csv


## Master Benchmark

In [ ]:
# --- MASTER BENCHMARK ---
results = []
print("Starting Final Benchmark...")

for filename in sorted(os.listdir(TEST_FOLDER_PATH)):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        path = os.path.join(TEST_FOLDER_PATH, filename)

        # 1. Baseline
        s = time.time()
        res_b = method_1_baseline(path)
        t_b = time.time() - s

        # 2. Smart Champion (V5)
        s = time.time()
        res_v5 = method_5_fast_ensemble(path)
        t_v5 = time.time() - s

        # We keep Pyramid/Weighted as 'Historical Data' if you need it,
        # but for the final table, focus on the 'Evolution'
        results.append({
            "File": filename,
            "Baseline": res_b,
            "Smart V5": res_v5,
            "B-Time": f"{t_b:.3f}s",
            "V5-Time": f"{t_v5:.3f}s",
        })

df_final = pd.DataFrame(results)
print(df_final.to_string(index=False))